In [33]:
# Alpaca for data
import alpaca_trade_api as api
from alpaca_trade_api.rest import TimeFrame

# pandas for analysis
import pandas as pd

# Plotly for charting
import plotly.graph_objects as go
import plotly.express as px

# Set default charting for pandas to plotly
pd.options.plotting.backend = "plotly"

# symbols we will be looking at
btc = "BTCUSD"
spy = "SPY"

# start dates and end dates for backtest
start_date = "2021-01-01"
end_date = "2021-10-20"

# time frame for backtests
timeframe = TimeFrame.Day

# periods for our SMA's
SMA_fast_period = 5
SMA_slow_period = 13

# Our API keys for Alpaca
API_SECRET = "0GOzkmT1F5W9rs8czJ90YAXKZrBn6jee3BgGo9le"
API_KEY = "PKNKABC4NQ6UURX5RU4R"
BASE_URL = 'https://paper-api.alpaca.markets'

# Setup instance of alpaca api
alpaca = api.REST(API_KEY, API_SECRET)

# # # Request historical bar data for SPY and BTC using Alpaca Data API
# for equities, use .get_bars
spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

# for crypto, use .get_crypto_bars, from multiple exchanges
btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df

# display crypto bar data
btc_data

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2021-01-01 06:00:00+00:00,BNCU,29255.71,29682.29,28707.56,29676.79,848.874030,29639,29316.444625
2021-01-01 06:00:00+00:00,CBSE,29413.29,29683.50,29039.00,29683.03,4431.837894,44893,29419.838845
2021-01-01 06:00:00+00:00,FTXU,29225.00,29631.00,28750.00,29631.00,25.076500,211,29330.826766
2021-01-02 06:00:00+00:00,BNCU,29678.34,34200.00,29555.99,33769.52,2144.592516,60152,31941.412694
2021-01-02 06:00:00+00:00,CBSE,32222.88,34205.00,32008.62,33779.48,7955.017558,87151,33089.671665
...,...,...,...,...,...,...,...,...
2021-10-19 05:00:00+00:00,FTXU,62482.00,64490.00,61395.00,64049.00,1486.986300,7100,63084.038259
2021-10-20 05:00:00+00:00,BNCU,64050.55,66996.28,63517.23,65050.04,2148.165472,68950,65519.685179
2021-10-20 05:00:00+00:00,CBSE,64058.65,66999.00,63525.00,65049.32,22479.505816,593526,65706.188033


In [34]:
# Keep data from only CBSE exchange
btc_data = btc_data[btc_data['exchange'] == 'CBSE'] 

# keep only the daily close data column
btc_data = btc_data.filter(['close'])

# rename our close column to BTC
btc_data.rename(columns={'close':'BTC'}, inplace=True)

# keep only the date part of our timestamp index
btc_data.index = btc_data.index.map(lambda timestamp : timestamp.date)

# Clean SPY data
spy_data = spy_data.filter(['close'])
spy_data.rename(columns={'close':'SPY'}, inplace=True)
spy_data.index = spy_data.index.map(lambda timestamp : timestamp.date)
spy_data

,SPY
2021-01-04,368.97
2021-01-05,371.40
2021-01-06,373.41
2021-01-07,379.13
2021-01-08,381.29
...,...
2021-10-14,442.50
2021-10-15,445.87
2021-10-18,447.19
2021-10-19,450.64


In [41]:
stock_name = 'SPY'
crypto_name = 'BTC'
data = btc_data.join(spy_data, how='outer')
data.index.name = 'date'
data = data.ffill()
data[stock_name+'_daily_return'] = data[stock_name].pct_change()
data[crypto_name+'_daily_return'] = data[crypto_name].pct_change()
a= 'SPY'
data[stock_name+'_return'] = data[stock_name+'_daily_return'].add(1).cumprod().sub(1)
data[crypto_name+'_return'] = data[crypto_name+'_daily_return'].add(1).cumprod().sub(1)

px.line(data,x=data.index, y=['SPY_return', 'BTC_return'])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3362425078.py, line 6)

In [8]:
# Computing the 5-day SMA and 13-day SMA
data['slow_SMA'] = data['BTC'].rolling(SMA_slow_period).mean()
data['fast_SMA'] = data['BTC'].rolling(SMA_fast_period).mean()

data.dropna(inplace=True)

data.plot(y=['BTC', 'slow_SMA', 'fast_SMA'])

In [10]:
# calculating when 5-day SMA crosses over 13-day SMA
crossover = data[(data['fast_SMA'] > data['slow_SMA']) \
       & (data['fast_SMA'].shift() < data['slow_SMA'].shift())]
                     
# calculating when 5-day SMA crosses unsw 13-day SMA
crossunder = data[(data['fast_SMA'] < data['slow_SMA']) \
        & (data['fast_SMA'].shift() > data['slow_SMA'].shift())]
# Plot green upward facing triangles at crossovers
fig1 = px.scatter(crossover, x=crossover.index, y='slow_SMA', \
                  color_discrete_sequence=['green'], symbol_sequence=[49])

# Plot red downward facing triangles at crossunders
fig2 = px.scatter(crossunder, x=crossunder.index, y='fast_SMA', \
                  color_discrete_sequence=['red'], symbol_sequence=[50])

# Plot slow sma, fast sma and price
fig3 = data.plot(y=['BTC', 'fast_SMA', 'slow_SMA'])

fig4 = go.Figure(data=fig1.data + fig2.data + fig3.data)
fig4.update_traces(marker={'size': 13})
fig4.show()

In [17]:
# New column for orders
crossover['order'] = 'buy'
crossunder['order'] = 'sell'


# Combine buys and sells into 1 data frame
orders = pd.concat([crossover[['BTC', 'order']], crossunder[['BTC','order']]]).sort_index()

# new dataframe with market data and orders merged
portfolio = pd.merge(data, orders, how='outer', left_index=True, right_index=True)
# "backtest" of our buy and hold strategies
portfolio['SPY_buy_&_hold'] = (portfolio['SPY_return'] + 1) * 10000
portfolio['BTC_buy_&_hold'] = (portfolio['BTC_return'] + 1) * 10000
# forward fill any missing data points in our buy & hold strategies 
# and forward fill BTC_daily_return for missing data points
portfolio[['BTC_buy_&_hold', 'SPY_buy_&_hold', 'BTC_daily_return',]] = \
                    portfolio[['BTC_buy_&_hold', 'SPY_buy_&_hold', 'BTC_daily_return']].ffill()
### Backtest of SMA crossover strategy
active_position = False
equity = 10000

# Iterate row by row of our historical data
for index, row in portfolio.iterrows():
    
    # change state of position
    if row['order'] == 'buy':
        active_position = True
    elif row['order'] == 'sell':
        active_position = False
    
    # update strategy equity
    if active_position:
        portfolio.loc[index, 'BTC_SMA_crossover'] = (row['BTC_daily_return'] + 1) * equity
        equity = portfolio.loc[index, 'BTC_SMA_crossover']
    else:
        portfolio.loc[index, 'BTC_SMA_crossover'] = equity
fig=px.line(portfolio[['BTC_SMA_crossover', 'BTC_buy_&_hold', 'SPY_buy_&_hold']], color_discrete_sequence=['green','blue', 'red'])

fig.show()

/var/folders/38/yrxk6p716_s1vjb47ppzhwsh0000gn/T/ipykernel_26284/463353079.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/38/yrxk6p716_s1vjb47ppzhwsh0000gn/T/ipykernel_26284/463353079.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
portfolio['BTC_SMA_daily_returns'] = portfolio['BTC_SMA_crossover'].pct_change()

mean_daily_return = portfolio['BTC_SMA_daily_returns'].mean()
std_daily_return = portfolio['BTC_SMA_daily_returns'].std()
spy_mean_daily_return = portfolio['SPY_daily_return'].mean()

trading_days = 252

daily_sharpe_ratio = (mean_daily_return - spy_mean_daily_return) / std_daily_return

annualized_sharpe_ratio = daily_sharpe_ratio * (trading_days ** 0.5)
annualized_sharpe_ratio

1.9066273413265438